In [ ]:
# User-editable variables here
jobName='Debra'

enableBucket=True
maxResolution='768,768'

In [ ]:
imageDir='/workspace/training'
configFile='/workspace/runpod_ai/scripts/LoraBasicSettings.json'

In [ ]:
!mkdir -p $imageDir
!mkdir -p $imageDir/logs
!mkdir -p $imageDir/images/tmp
!mkdir -p $imageDir/output
!mkdir -p $imageDir/reg

In [ ]:
#Pause here
print('Upload training pictues to '+imageDir+'/images/tmp')

In [ ]:
import os

directory = imageDir+'/images/tmp'  # Specify the directory path
numFiles = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# min 100/image, min 1500/epoch
numSteps = 1500/numFiles
if numSteps < 100:
    numSteps = 100
print('Steps per image:  '+str(numSteps))
print('Steps per epoch:  '+str(numSteps*numFiles))

newDirName=str(numSteps)+'_'+jobName
%cd $imageDir/images
!mv tmp $newDirName
print('Folder to caption:  '+imageDir+'/images/'+newDirName)

In [ ]:
origConfigFile=configFile+'.orig'
!cp $origConfigFile $configFile
!sed -i 's/\"output_name\": \"latest\",/\"output_name\": \"{jobName}\",/' $configFile
print('Configuration file:  '+configFile)

In [ ]:
#Pause here
# Load config file
# Run 'Utilities/WD14 Captioning'

In [ ]:
print('Folder to caption:  '+imageDir+'/images/'+newDirName)

In [ ]:
# Start training in kohya_ss UI.